In [1]:
import numpy as np 
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import LabelEncoder, MinMaxScaler , OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import FeatureUnion, Pipeline 
from sklearn.preprocessing import PowerTransformer
from sklearn.experimental import enable_iterative_imputer

from sklearn.impute import IterativeImputer

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from scipy.stats import skew, norm
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax
warnings.filterwarnings('ignore')
%matplotlib inline

C:\Users\Kostya\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [3]:
train = pd.read_csv('train.csv').drop('Id',axis = 1 )
test = pd.read_csv('test.csv').drop('Id',axis = 1 )
test_Y = pd.read_csv('sample_submission.csv').drop("Id", axis = 1)

In [4]:
train["SalePrice"] = np.log1p(train["SalePrice"])
y_test = np.log1p(test_Y)

In [5]:
train.drop(train[(train['OverallQual']<5) & (train['SalePrice']>200000)].index, inplace=True)
train.drop(train[(train['GrLivArea']>4500) & (train['SalePrice']<300000)].index, inplace=True)
train.reset_index(drop=True, inplace=True)

In [123]:
train_labels = train['SalePrice'].reset_index(drop=True)
train_features = train.drop(['SalePrice'], axis=1)
test_features = test

# Объединяем тестовую и тренировочную выборку
all_features = pd.concat([train_features, test_features]).reset_index(drop=True)
all_features.shape

(2917, 79)

In [124]:
def percent_missing(df):
    data = pd.DataFrame(df)
    df_cols = list(pd.DataFrame(data))
    dict_x = {}
    for i in range(0, len(df_cols)):
        dict_x.update({df_cols[i]: round(data[df_cols[i]].isnull().mean()*100,2)})
    
    return dict_x

missing = percent_missing(all_features)
df_miss = sorted(missing.items(), key=lambda x: x[1], reverse=True)
print('Percent of missing data')
df_miss[0:10]

Percent of missing data


[('PoolQC', 99.69),
 ('MiscFeature', 96.4),
 ('Alley', 93.21),
 ('Fence', 80.43),
 ('FireplaceQu', 48.68),
 ('LotFrontage', 16.66),
 ('GarageYrBlt', 5.45),
 ('GarageFinish', 5.45),
 ('GarageQual', 5.45),
 ('GarageCond', 5.45)]

In [125]:
all_features['MSSubClass'] = all_features['MSSubClass'].apply(str)
all_features['YrSold'] = all_features['YrSold'].astype(str)
all_features['MoSold'] = all_features['MoSold'].astype(str)
all_features['YearRemodAdd'] = all_features['YearRemodAdd'].astype(str)
all_features['YearBuilt'] = all_features['YearBuilt'].astype(str)

In [126]:
def handle_missing(features):
    # the data description states that NA refers to typical ('Typ') values
    features['Functional'] = features['Functional'].fillna('Typ')
    # Replace the missing values in each of the columns below with their mode
    features['Electrical'] = features['Electrical'].fillna("SBrkr")
    features['KitchenQual'] = features['KitchenQual'].fillna("TA")
    features['Exterior1st'] = features['Exterior1st'].fillna(features['Exterior1st'].mode()[0])
    features['Exterior2nd'] = features['Exterior2nd'].fillna(features['Exterior2nd'].mode()[0])
    features['SaleType'] = features['SaleType'].fillna(features['SaleType'].mode()[0])
    features['MSZoning'] = features.groupby('MSSubClass')['MSZoning'].transform(lambda x: x.fillna(x.mode()[0]))
    
    # the data description stats that NA refers to "No Pool"
    features["PoolQC"] = features["PoolQC"].fillna("None")
    # Replacing the missing values with 0, since no garage = no cars in garage
    for col in ('GarageYrBlt', 'GarageArea', 'GarageCars'):
        features[col] = features[col].fillna(0)
    # Replacing the missing values with None
    for col in ['GarageType', 'GarageFinish', 'GarageQual', 'GarageCond']:
        features[col] = features[col].fillna('None')
    # NaN values for these categorical basement features, means there's no basement
    for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):
        features[col] = features[col].fillna('None')
        
    # Group the by neighborhoods, and fill in missing value by the median LotFrontage of the neighborhood
    features['LotFrontage'] = features.groupby('Neighborhood')['LotFrontage'].transform(lambda x: x.fillna(x.median()))

    # We have no particular intuition around how to fill in the rest of the categorical features
    # So we replace their missing values with None
    objects = []
    for i in features.columns:
        if features[i].dtype == object:
            objects.append(i)
    features.update(features[objects].fillna('None'))
        
    # And we do the same thing for numerical features, but this time with 0s
    numeric_dtypes = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    numeric = []
    for i in features.columns:
        if features[i].dtype in numeric_dtypes:
            numeric.append(i)
    features.update(features[numeric].fillna(0))    
    return features

all_features = handle_missing(all_features)

In [127]:
# Fetch all numeric features
numeric_dtypes = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numeric = []
for i in all_features.columns:
    if all_features[i].dtype in numeric_dtypes:
        numeric.append(i)

In [128]:
# Find skewed numerical features
skew_features = all_features[numeric].apply(lambda x: skew(x)).sort_values(ascending=False)

high_skew = skew_features[skew_features > 0.5]
skew_index = high_skew.index

print("There are {} numerical features with Skew > 0.5 :".format(high_skew.shape[0]))
skewness = pd.DataFrame({'Skew' :high_skew})
skew_features.head(10)

There are 25 numerical features with Skew > 0.5 :


MiscVal          21.939672
PoolArea         17.688664
LotArea          13.109495
LowQualFinSF     12.084539
3SsnPorch        11.372080
KitchenAbvGr      4.300550
BsmtFinSF2        4.144503
EnclosedPorch     4.002344
ScreenPorch       3.945101
BsmtHalfBath      3.929996
dtype: float64

In [129]:
# Normalize skewed features
for i in skew_index:
    all_features[i] = boxcox1p(all_features[i], boxcox_normmax(all_features[i] + 1))

In [130]:
for col in all_features.columns[all_features.dtypes != 'O']:
    all_features[col] = mmx.fit_transform(all_features[col].values.reshape(-1,1))

In [131]:
all_features

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,0.241215,0.315220,Pave,None,Reg,Lvl,AllPub,Inside,...,0.0,0.0,None,None,None,0.000000,2,2008,WD,Normal
1,20,RL,0.305987,0.338667,Pave,None,Reg,Lvl,AllPub,FR2,...,0.0,0.0,None,None,None,0.000000,5,2007,WD,Normal
2,60,RL,0.254622,0.368183,Pave,None,IR1,Lvl,AllPub,Inside,...,0.0,0.0,None,None,None,0.000000,9,2008,WD,Normal
3,70,RL,0.218294,0.337702,Pave,None,IR1,Lvl,AllPub,Corner,...,0.0,0.0,None,None,None,0.000000,2,2006,WD,Abnorml
4,60,RL,0.322395,0.413082,Pave,None,IR1,Lvl,AllPub,FR2,...,0.0,0.0,None,None,None,0.000000,12,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2912,160,RM,0.000000,0.062763,Pave,None,Reg,Lvl,AllPub,Inside,...,0.0,0.0,None,None,None,0.000000,6,2006,WD,Normal
2913,160,RM,0.000000,0.059249,Pave,None,Reg,Lvl,AllPub,Inside,...,0.0,0.0,None,None,None,0.000000,4,2006,WD,Abnorml
2914,20,RL,0.589806,0.478793,Pave,None,Reg,Lvl,AllPub,Inside,...,0.0,0.0,None,None,None,0.000000,9,2006,WD,Abnorml
2915,85,RL,0.227553,0.354243,Pave,None,Reg,Lvl,AllPub,Inside,...,0.0,0.0,None,MnPrv,Shed,0.653989,7,2006,WD,Normal


In [132]:
all_features['MSSubClass'] = all_features['MSSubClass'].apply(float)
all_features['YrSold'] = all_features['YrSold'].astype(float)
all_features['MoSold'] = all_features['MoSold'].astype(float)
all_features['YearRemodAdd'] = all_features['YearRemodAdd'].astype(float)
all_features['YearBuilt'] = all_features['YearBuilt'].astype(float)

In [133]:
all_features = pd.get_dummies(all_features).reset_index(drop=True)
all_features.shape

(2917, 302)

In [134]:
all_features = all_features.loc[:,~all_features.columns.duplicated()]

In [135]:
all_features.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,60.0,0.241215,0.315220,0.666667,0.536679,2003.0,2003.0,0.388013,0.291583,0.0,...,0,0,0,1,0,0,0,0,1,0
1,20.0,0.305987,0.338667,0.555556,0.888355,1976.0,1976.0,0.000000,0.367659,0.0,...,0,0,0,1,0,0,0,0,1,0
2,60.0,0.254622,0.368183,0.666667,0.536679,2001.0,2002.0,0.354782,0.223447,0.0,...,0,0,0,1,0,0,0,0,1,0
3,70.0,0.218294,0.337702,0.666667,0.536679,1915.0,1970.0,0.000000,0.125026,0.0,...,0,0,0,1,1,0,0,0,0,0
4,60.0,0.322395,0.413082,0.777778,0.536679,2000.0,2000.0,0.507233,0.276422,0.0,...,0,0,0,1,0,0,0,0,1,0


In [136]:
all_features.columns[all_features.isna().sum()>0]

Index([], dtype='object')

In [137]:
X_train = all_features.iloc[:len(train_labels), :]
X_test = all_features.iloc[len(train_labels):, :]
X_train.shape, train_labels.shape, X_test.shape

((1458, 302), (1458,), (1459, 302))

In [138]:
LR = LinearRegression()
LR.fit(X_train,train_labels)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [139]:

y_pred = LR.predict(X_test)

In [140]:
y_pred

array([11.69962054, 12.02073538, 12.15140855, ..., 12.10295286,
       11.69327185, 12.29481334])

In [142]:
LR.score(X_test,y_test.values)

-34.18974633128346